<a href="https://colab.research.google.com/github/BharatXBhalla/International-GHG-Emission-API/blob/main/Database/Preprocess_Sqlite_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
import sqlite3
from sqlite3 import Error

In [225]:
sqlInfo={
    "tables":["CO2","GHG_Indirect_CO2","GHG","HFC","CH4","HF3","N2O","PFCs","SF6","HFC_Mix_PFC"],
    "tableQueries":[
'''CREATE TABLE `Countries` (
  `id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `name` varchar(255),
  `start_year` INTEGER,
  `end_year` INTEGER
);''',

'''CREATE TABLE `CO2` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `GHG_Indirect_CO2` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `GHG` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `HFC` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `CH4` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `HF3` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `N2O` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `PFCs` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `SF6` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''',

'''CREATE TABLE `HFC_Mix_PFC` (
  `country_id` INTEGER,
  `year` INTEGER,
  `value` real,
  FOREIGN KEY(`country_id`) REFERENCES Countries(id)
);''']
}

In [226]:
preprocess_data_url="https://raw.githubusercontent.com/BharatXBhalla/International-GHG-Emission-API/main/EDA/Processed_GHG_Data.csv"

In [227]:
database_file_name="ghg.db"

In [228]:
def createConnection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def createTable(connection_obj,query):
  # cursor object
  cursor_obj = connection_obj.cursor()

  # Drop the GEEK table if already exists.
  #cursor_obj.execute("DROP TABLE IF EXISTS %s"%tablename)


  result=cursor_obj.execute(query)
  print(result)
  print("Table is Ready")

def selectAllTable(connection_obj,table_name,print_rows=False):
  cur =connection_obj.cursor()
  cur.execute("SELECT * FROM %s"%(table_name))
  rows = cur.fetchall()
  if print_rows==True:
    for row in rows:
      print(row)
  return rows

def insertTable(connection_obj,table_name,value_name,values):

  # Creating a cursor object using the
  # cursor() method
  cursor = connection_obj.cursor()
  # Queries to INSERT records.
  cursor.execute('''INSERT INTO %s %s VALUES %s;'''%(table_name,value_name,values))
  # Commit your changes in the database	
  connection_obj.commit()
    

In [229]:
connection_obj = createConnection(database_file_name)

In [230]:
for table in sqlInfo["tableQueries"]:
  createTable(connection_obj,table)

Table is Ready
Table is Ready
Table is Ready
Table is Ready
Table is Ready
Table is Ready
Table is Ready
Table is Ready
Table is Ready
Table is Ready
Table is Ready


In [231]:
#insertTable(connection_obj,"CO2","(country_id,year,value)","('%d',%d,%f)"%(1,2009,93838.99) )

In [232]:
import pandas as pd

In [233]:
df = pd.read_csv(preprocess_data_url).iloc[:,1:]

In [234]:
df.head()

,country_or_area,year,value,category,short_category
0,Australia,2014,393126.946994,carbon_dioxide_co2_emissions_without_land_use_...,CO2
1,Australia,2013,396913.936530,carbon_dioxide_co2_emissions_without_land_use_...,CO2
2,Australia,2012,406462.847704,carbon_dioxide_co2_emissions_without_land_use_...,CO2
3,Australia,2011,403705.528314,carbon_dioxide_co2_emissions_without_land_use_...,CO2
4,Australia,2010,406200.993184,carbon_dioxide_co2_emissions_without_land_use_...,CO2


In [235]:
df.short_category.unique()

array(['CO2', 'GHG_Indirect_CO2', 'GHG', 'HFC', 'CH4', 'HF3', 'N2O',
       'PFCs', 'SF6', 'HFC_Mix_PFC'], dtype=object)

In [236]:
def insertCountry(df,database_file_name):
  connection_obj = createConnection(database_file_name)
  g=df.groupby("country_or_area")
  startYear=g.year.min()
  endYear=g.year.max()
  for country in df["country_or_area"].unique():
    insertTable(connection_obj,"Countries","(name,start_year,end_year)","('%s',%d,%d)"%(country,startYear[country],endYear[country]) )
  connection_obj.close()

In [237]:
def insertEmissions(df,database_file_name,country_mapping):
  connection_obj = createConnection(database_file_name)
  for i in df.values:
    insertTable(connection_obj,i[4],"(country_id,year,value)","('%d',%d,%f)"%(country_mapping[i[0]],i[1],i[2]) )
  
  connection_obj.close()

In [238]:
insertCountry(df,database_file_name)

In [239]:
countries=dict([(i[1],i[0]) for i in selectAllTable(connection_obj,"Countries")])

In [240]:
insertEmissions(df,database_file_name,countries)

In [241]:
df["short_category"].value_counts()

CO2                 1074
GHG                 1074
CH4                 1074
N2O                 1074
SF6                 1032
HFC                  975
GHG_Indirect_CO2     949
PFCs                 831
HF3                  248
HFC_Mix_PFC           75
Name: short_category, dtype: int64

In [242]:
for i in df["short_category"].value_counts().index:
  f=selectAllTable(connection_obj,i)
  print(len(f))

1074
1074
1074
1074
1032
975
949
831
248
75


In [243]:
connection_obj.close()